In [ ]:
import keras
import tensorflow as tf
import sys
import os
from keras.layers import *
from keras import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras.backend as K
import pandas as pd 
import numpy as np
import h5py
import scipy.io

#scikit learn
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
%matplotlib inline

from keras import losses
from sklearn.metrics import confusion_matrix,recall_score,matthews_corrcoef,roc_curve,roc_auc_score,auc
from sklearn.metrics import precision_recall_fscore_support
from scipy import interp
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve, auc
import math
from operator import truediv
from sklearn.metrics import confusion_matrix
import itertools
import pickle
from keras.constraints import unit_norm
from keras.constraints import max_norm

In [ ]:
cfm = 0
fold= 10

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

for j in range(fold):   
    
    # folder locations
    model_name='Yeast_Model6540_'+str(j)
    result_folder= 'Results/'
    model_results_folder=result_folder+model_name+ '/'
    best_weights = model_results_folder + 'best_weights.h5'
    model_arch = model_results_folder + 'model.json'
    
    # Loading model json file 
    json_file = open(model_results_folder+'model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
        
    test_X = np.load('folds/'+str(j)+'_x_test.npy')
    test_y = np.load('folds/'+str(j)+'_y_test.npy')
    
    # Loading Model weights
    model.load_weights(best_weights)

    # Prediction of Test data
    y_score=model.predict(test_X)
    average_precision = average_precision_score(test_y, y_score)
    fpr, tpr, _ = roc_curve(test_y, y_score)
    roc_auc = auc(fpr, tpr)
    tn, fp, fn, tp= confusion_matrix(test_y, np.round(y_score)).ravel()
    
    print('_______')
    print('Fold',j)
    print('_______')
    
    # Calculating evaluation metrics based on formulars
    
    print('sn',np.round(float((tp*100))/(tp+fn),4))
    print('sp',np.round(float((tn*100))/(tn+fp),4))
    print('ACC',np.round((truediv(tp+tn,(tp+fn+tn+fp)))*100,2))
    numerator1=(tp * tn - fp * fn)
    denominator1 =math.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    mcc1=np.round((truediv(numerator1,denominator1+K.epsilon()))*100,2)
    print('MCC',mcc1)
    print('auROC',np.round((roc_auc)*100,2))
    
    
    # Calculating auROC's of each fold and appending on aucs to make an average of all 10 folds. 
    fpr_rf, tpr_rf, thresholds_rf=metrics.roc_curve(test_y,y_score)

    tprs.append(interp(mean_fpr, fpr_rf, tpr_rf))
    tprs[-1][0] = 0.0
    auc_rf = auc(fpr_rf, tpr_rf)
    aucs.append(auc_rf)
    
    # Calculating confusion matrix 
    confusion=confusion_matrix(test_y, np.round(y_score))
    cfm=cfm+confusion

cfm=cfm/10.
       

cfm=cfm.astype(int)
tp=cfm[1][1]
tn=cfm[0][0]
fp=cfm[0][1]
fn=cfm[1][0]
tp=np.float(tp)
tn=np.float(tn)
fp=np.float(fp)
fn=np.float(fn)

print('------------')
print('RESUTLS')
print('------------')
# Taking the Average MCC of all 10 folds
numerator=(tp * tn - fp * fn)
denominator =math.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
mcc=np.round((truediv(numerator,denominator+K.epsilon()))*100,2)
print('MCC',mcc)

#Taking the Average ACCURACY of all 10 folds 
acc=np.round((truediv(tp+tn,(tp+fn+tn+fp)))*100,2)
print('ACC',acc)

#Taking the Average Sensitivity of all 10 folds 
sn=np.round(float((tp*100))/(tp+fn),2)
print('sn',sn)
      
#Taking the Average Specificity of all 10 folds 
sp=np.round(float((tn*100))/(tn+fp),2)
print('sp',sp)

#Taking the Average auROC's of all 10 folds 
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
print('auROC',mean_auc)